In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import seaborn as sns
from datetime import datetime 
import import_ipynb 
from lib_final import *

from pandasql import sqldf 

In [47]:
df = pd.read_csv('clean_data.csv')
df.head()

,Id,Title,Price(Triệu/tháng),Square(m²),District,City,Date
0,478324,Cho thuê phòng để ở tại 108/64A1 Trần Quang Di...,3.5,18.0,Quận 3,Hồ Chí Minh,20/06/2020
1,728993,Phòng vừa xây xong thoáng mát gần các trường Đ...,3.3,25.0,Quận 7,Hồ Chí Minh,17/10/2018
2,781296,Cần share lại gấp phòng trọ mới thuê tại đường...,3.0,15.0,Quận 10,Hồ Chí Minh,05/10/2018
3,781448,Chính chủ cho thuê căn hộ dịch vụ gần sân bay ...,7.0,27.0,Tân Bình,Hồ Chí Minh,06/10/2018
4,789361,"Phòng cho thuê giá rẻ, Quận Gò Vấp gần chung c...",1.7,12.0,Gò Vấp,Hồ Chí Minh,09/10/2018


In [48]:
# Lọc ra bộ dữ liệu chỉ gồm phòng trọ
df = title_clean('Title', df)

# Xóa những cột mà Price = null và Square = null
df = df[df["Price(Triệu/tháng)"].notna()]
df = df[df["Square(m²)"].notna()]
df.head()

,Id,Title,Price(Triệu/tháng),Square(m²),District,City,Date
0,781296,cần share lại gấp phòng trọ mới thuê tại đường...,3.0,15.0,Quận 10,Hồ Chí Minh,05/10/2018
1,834229,cho thuê phòng trọ quận bình thạnh gần ngã tư ...,1.5,16.0,Bình Thạnh,Hồ Chí Minh,11/10/2018
2,842926,"phòng trọ cách mạng tháng 8, quận 3",4.0,20.0,Quận 3,Hồ Chí Minh,01/10/2018
3,847848,"cho thuê phòng trọ khu bàu cát, p14, q. tân bì...",5.0,25.0,Tân Bình,Hồ Chí Minh,01/10/2018
4,849068,"phòng trọ hẻm 467 lê đức thọ , phường 16, quận...",1.8,20.0,Gò Vấp,Hồ Chí Minh,18/10/2018


- Câu hỏi: Sinh viên nên thuê trọ vào tháng nào nhất ở một quận cụ thể?  
- Hướng trả lời:
  - Chỉ số x = giá phòng trọ/1m2/1 tháng
  - Tháng nên thuê phòng trọ là tháng có chỉ số x trung bình nhỏ nhất (tính từ năm 2018 đến 2022).


In [52]:
df["Date"].unique()

array(['05/10/2018', '11/10/2018', '01/10/2018', '18/10/2018',
       '02/10/2018', '23/10/2018', '24/10/2018', '03/10/2018',
       '08/10/2018', '12/10/2018', '20/10/2018', '21/10/2018',
       '22/10/2018', '25/10/2018', '26/10/2018', '28/10/2018',
       '29/10/2018', '01/11/2018', '03/11/2018', '07/11/2018',
       '22/11/2018', '26/11/2018', '06/12/2018', '08/12/2018',
       '09/12/2018', '12/12/2018', '14/12/2018', '16/12/2018',
       '18/01/2019', '18/12/2018', '27/02/2019', '21/12/2018',
       '23/12/2018', '24/12/2018', '25/12/2018', '02/01/2019',
       '04/01/2019', '06/01/2019', '08/01/2019', '11/01/2019',
       '12/01/2019', '31/01/2019', '21/02/2019', '22/02/2019',
       '01/03/2019', '03/03/2019', '04/03/2019', '05/03/2019',
       '06/03/2019', '07/03/2019', '13/03/2019', '16/03/2019',
       '17/03/2019', '18/03/2019', '19/03/2019', '21/03/2019',
       '22/03/2019', '23/03/2019', '25/03/2019', '27/03/2019',
       '29/03/2019', '02/04/2019', '09/04/2019', '16/04

In [56]:
# Thêm cột Month, Year
#df['Year'] = df['Date'].str.slice(0,4)
#df['Month'] = df['Date'].str.slice(5,7)
df["Date"] = pd.to_datetime(df["Date"],format = "%d/%m/%Y ",errors = 'coerce')
year = df['Date'].array
df['Year'] = year[:].year
month = df['Date'].array
df['Month'] = month[:].month

In [57]:
# Thêm cột chỉ số x 
df["X"] = df["Price(Triệu/tháng)"]/df["Square(m²)"]
df = df[["X", "Price(Triệu/tháng)", "Square(m²)", "District", "Month", "Year"]]
df[df["District"] == "Bình Chánh"]
df.rename(columns={'Price(Triệu/tháng)':'Price'},inplace = True)

In [58]:
# pd.set_option("display.max_rows", None)
# Gom cụm theo quận, năm, tháng để xem dữ liệu
query = "SELECT df.X,df.Price, df.District, df.Month, df.Year FROM df GROUP BY District, Year, Month"
sqldf(query, globals())

,X,Price,District,Month,Year
0,0.066667,2.0,Bình Chánh,2,2020
1,0.066667,2.0,Bình Chánh,12,2020
2,0.080000,2.0,Bình Chánh,4,2021
3,0.014000,1.4,Bình Chánh,4,2022
4,0.093750,1.5,Bình Thạnh,10,2018
...,...,...,...,...,...
308,0.125000,4.0,Tân Phú,11,2020
309,0.090000,2.7,Tân Phú,3,2021
310,0.075000,4.5,Tân Phú,6,2021
311,0.120000,1.8,Tân Phú,11,2021


In [59]:
""" 
- Tạo câu truy vấn: gom các tháng của cùng 1 quận lại với nhau (của tất cả các năm từ 2018 - 2022)
- Ví dụ kết quả truy vấn mong đợi:
        ** BẢNG DATA BAN ĐẦU **
 + District    Year     Month   Price    ...
 +  A          2018       5      0.1     ...
 +  A          2018       8      0.2     ...
 +  A          2019       5      0.8     ... 
 +  A          2020       8      0.1     ... 
 +  A          2021       8      0.1     ...

        ** BẢNG KẾT QUẢ **
 + District     Month    Price   ...  (không có cột Year)
 + A              5       0.9    
 + A              8       0.4    ...
"""
# query = "SELECT DISTINCT a.X, a.Price, a.Square, a.District, a.Month \
#         FROM df a \
#         LEFT JOIN df2 b \
#         ON a.Month = b.Month AND a.District = b.District"
# ans = sqldf(query, globals())
# ans[ans["District"] == "Bình Chánh"]

' \n- Tạo câu truy vấn: gom các tháng của cùng 1 quận lại với nhau (của tất cả các năm từ 2018 - 2022)\n- Ví dụ kết quả truy vấn mong đợi:\n        ** BẢNG DATA BAN ĐẦU **\n + District    Year     Month   Price    ...\n +  A          2018       5      0.1     ...\n +  A          2018       8      0.2     ...\n +  A          2019       5      0.8     ... \n +  A          2020       8      0.1     ... \n +  A          2021       8      0.1     ...\n\n        ** BẢNG KẾT QUẢ **\n + District     Month    Price   ...  (không có cột Year)\n + A              5       0.9    \n + A              8       0.4    ...\n'

In [60]:
"""       ** BẢNG DATA BAN ĐẦU **
 + District    Year     Month   Price    ...
 +  A          2018       5      0.1     ...
 +  A          2018       8      0.2     ...
 +  A          2019       5      0.8     ... 
 +  A          2020       8      0.1     ... 
 +  A          2021       8      0.1     ...

        ** BẢNG KẾT QUẢ **
 + District     Month    Price   ...  (không có cột Year)
 + A              5       0.9    
 + A              8       0.4    ... (Giả sử tính sum())
"""
result = df.groupby(['District', 'Month']).mean() # tính trung bình chỉ số X của mỗi quận, trên từng tháng, tính từ năm 2018 - 2022
result

X     Price  Square(m²)     Year
District   Month                                         
Bình Chánh 2      0.066667  2.000000   30.000000  2020.00
           4      0.047000  1.700000   62.500000  2021.50
           12     0.066667  2.000000   30.000000  2020.00
Bình Thạnh 1      0.210909  3.440000   17.400000  2020.20
           2      0.123750  2.425000   19.250000  2021.75
...                    ...       ...         ...      ...
Tân Phú    6      0.094318  3.500000   41.000000  2020.50
           8      0.166667  4.000000   24.000000  2020.00
           10     0.108316  2.530000   21.875000  2019.25
           11     0.120333  2.900000   24.000000  2020.00
           12     0.118148  3.033333   25.666667  2019.00

[170 rows x 4 columns]

In [61]:
result.loc["Bình Chánh", :]["X"].idxmin()

4

In [62]:
# Lấy chỉ số X của từng tháng
city_list = df["District"].unique()
city_list
for city in city_list:
    print("Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận %s: Tháng"%city,  
    pd.to_numeric(result.loc[city, :]["X"].idxmin()))


Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Quận 10: Tháng 2
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Bình Thạnh: Tháng 3
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Quận 3: Tháng 5
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Tân Bình: Tháng 2
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Gò Vấp: Tháng 5
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Quận 7: Tháng 2
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Bình Tân: Tháng 8
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Quận 2: Tháng 6
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Hóc Môn: Tháng 10
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Phú Nhuận: Tháng 9
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Quận 1: Tháng 7
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ số X) ở quận Quận 4: Tháng 3
Tháng có giá thuê trọ nhỏ nhất trên 1m2 (hay chỉ s